# Twins Example

In this notebook, code is provided for the Twin Study example presented in the paper "A Fisher Scoring approach for crossed multiple-factor Linear Mixed Models". The data used in this example was taken from the Wu-Minn Human Connectome project [(Van Essen et al. (2013))](https://pubmed.ncbi.nlm.nih.gov/23684880/) and is not freely available. To obtain the data, see the [Wu-Minn HCP cohort website](https://www.humanconnectome.org/study/hcp-young-adult/document/wu-minn-hcp-consortium-open-access-data-use-terms). Further detail on preprocessing is provided below.

## Python Imports

In [ ]:
# Package imports
import numpy as np
import scipy
import os
import sys
import pandas as pd
import time

# Import modules from elsewhere in the repository.
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(os.path.join(module_path,"src","TwinExample"))
    sys.path.append(os.path.join(module_path,"src","lib"))
    
from genTestDat import genTestData2D, prodMats2D
from est2d import *
from npMatrix2d import *
from ACE import *
from scipy.optimize import minimize
from scipy import stats

## Preprocessing

As mentioned above, the data for this example must be obtained from [Wu-Minn HCP cohort website](https://www.humanconnectome.org/study/hcp-young-adult/document/wu-minn-hcp-consortium-open-access-data-use-terms). Once the application process has been completed, access will be granted to $2$ files; the unrestricted (non-sensitive) data and the restricted (sensitive) data. Due to the regulation surrounding these files, we cannot provide either here. To run the following code, please first apply for access to the data and download the data. 

To begin we first preprocess the data using the Matlab function `hcp2blocks.m` and the below Matlab code. This code is borrowed from the work of [Winkler A. et al (2015)](https://www.sciencedirect.com/science/article/pii/S105381191500508X) and the `hcp2blocks.m` function can be found in the ``src/TwinExample`` folder in this repository. This is the only time Matlab code is used in this work. The below code sorts family units by family structure type and saves the result in the filepath given by the variable `outputfile`.

```
% Add the path to the `hcp2blocks` function.
addpath(% Enter path here...)

% Add the path to the restricted data file here.
restrfile = % Enter restricted data path here...

% The below path describes where some of the output of the function will be saved. We don't actually
% use this file so enter any path you like here and feel free to delete this file after the function
% has been run.
blocksfile = % Enter a name for the csv file that will be output...


% Run the hcp2blocks function
[EB,tabout] = hcp2blocks(restrfile,blocksfile,true,[],true)

% We now name the columns in the tabout table.
cHeader = {'Subject' 'Mother_ID' 'Father_ID' 'sibtype','familyID', 'familyType', 'ignore'}; 
commaHeader = [cHeader;repmat({','},1,numel(cHeader))]; %insert commaas
commaHeader = commaHeader(:)';
textHeader = cell2mat(commaHeader); %cHeader in text with commas

% Specify an output file for the new table
outputfile = # Path to family type table...

% Write the header column headers to the file
fid = fopen(outputfile,'w'); 
fprintf(fid,'%s\n',textHeader)
fclose(fid)

%write data to end of file
dlmwrite(outputfile,tabout,'-append','precision',8);
```

After running the above code, you should now have $3$ files that are used throughout the rest of this notebook. These are:

 - The unrestricted data file (downloaded from HCP)
 - The restricted data file (downloaded from HCP)
 - The family type file (created by the above Matlab code as `outputfile`)
 
Each of these must now be loaded into memory below:

In [ ]:
# Read in the unrestricted data file
unrestricted = pd.read_csv(# Path to unrestricted data file...)

# Read in the restricted data file
restricted = pd.read_csv(# Path to restricted data file...)

# Read in family type data
famTypewoDZ = pd.read_csv(# Path to family type file...)

We now reduce the data to only the variables we need and combine all variables into one table.

In [ ]:
# Remove columns not of interest from the family type data.
reducedData = famTypewoDZ[['Subject','familyType','familyID']].sort_values(by=['familyType','familyID'])

# Reduce the restricted dataset to the variables we need
reducedRestricted = restricted[['Subject','Age_in_Yrs','HasGT','ZygosityGT','ZygositySR', 'Family_ID','Mother_ID','Father_ID']]

# Make a working table
newTab = pd.merge(reducedData, reducedRestricted, on='Subject')

# Get a table where every pair of parents has a corresponding count/number of children
parentTable = newTab.groupby(['Mother_ID','Father_ID']).size().sort_values(ascending=True).reset_index().rename(columns={0:'ParentCounts'})

# Add the parent counts to the new table
newTab = pd.merge(newTab,parentTable,on=['Mother_ID','Father_ID'])

# Reduce the unrestricted dataset to the variables we need
reducedUnrestricted = unrestricted[['Subject','Gender','ReadEng_Unadj','PSQI_Score']]

# Add the unrestricted data to our table and drop na values
newTab = pd.merge(newTab,reducedUnrestricted,on=['Subject']).dropna()

# Recode the gender variable to 0 and 1.
newTab['Gender']=newTab['Gender'].replace(['F','M'],[0,1])

# Add age and sex interaction
newTab['Age:Sex']=newTab[['Age_in_Yrs']].values*newTab[['Gender']].values

# Apply the appropriate sort
newTab=newTab.sort_values(by=['familyType','familyID','ParentCounts','ZygosityGT','ZygositySR'],ascending=[True,True,False,True,False])

## Check family types have been encoded correctly

The `MATLAB` code mentioned above was designed to calculate the family type for all family units in the HCP dataset (i.e. it categorizes family units by the number of siblings and types of siblings they contain). However, when we decide which variables we wish to use, subjects with missing entries for those variables are dropped from the analysis. This means that some family units may now be ``a person short``. For this reason, the below code has been written to loop through all family units and check they have been assigned to the correct family type. If not it assigns them a new family type.

In [ ]:
# Work out the unique types of family
UniqueFamilyTypes, idx = np.unique(newTab[['familyType']], return_index=True)
UniqueFamilyTypes = UniqueFamilyTypes[np.argsort(idx)]

# Number of grouping factors r
r = len(UniqueFamilyTypes)

# Loop through each family type (these are our factors)
for k in np.arange(r):

    # Work out which family type we're looking at
    uniqueType = UniqueFamilyTypes[k]

    # Get the table of these families
    familyTypeTable = newTab[newTab['familyType']==uniqueType]

    # Get a list of all family IDs in this category
    uniqueFamilyIDs = np.unique(familyTypeTable[['familyID']])

    # Loop through each family and work out the number of family members
    noFamMem = 0
    for j in np.arange(len(uniqueFamilyIDs)):

        # Get the ID for this family
        famID = uniqueFamilyIDs[j]

        # Get the table for this ID
        famTable = familyTypeTable[familyTypeTable['familyID']==famID]

        # Work out the number of subjects in this family
        noFamMem = np.maximum(noFamMem, famTable.shape[0])

    # Loop through each family and check they have all family members
    for j in np.arange(len(uniqueFamilyIDs)):

        # Get the ID for this family
        famID = uniqueFamilyIDs[j]

        # Get the table for this ID
        famTable = familyTypeTable[familyTypeTable['familyID']==famID]

        # If we don't have all subjects drop this family (we could recalculate
        # the family indexes... but this is only an illustrative example).
        if noFamMem > famTable.shape[0]:

            # Drop the familys that are now missing subjects.
            #newTab = newTab.drop(newTab[newTab.familyID == famID].index)

            newTab['familyType'][newTab.familyID == famID] = np.amax(UniqueFamilyTypes)+1
            UniqueFamilyTypes = np.append(UniqueFamilyTypes,np.amax(UniqueFamilyTypes)+1)

# Recalculate the unique types of family
UniqueFamilyTypes, idx = np.unique(newTab[['familyType']], return_index=True)
UniqueFamilyTypes = UniqueFamilyTypes[np.argsort(idx)]

# Recalculate number of grouping factors r
r = len(UniqueFamilyTypes)

## Variables representing the LMM

We now must construct $X$ and $Y$ for the LMM of the form $Y=X\beta+Zb+\epsilon$. $Z$ does not need to be constructed as, for the Twin dataset, $Z$ is just the identity matrix.

In [ ]:
# Construct X
X = newTab[['Age_in_Yrs','Gender','Age:Sex','PSQI_Score']].values 

# Add an intercept to X
X = np.concatenate((np.ones((X.shape[0],1)),X),axis=1)

# Construct Y
Y = newTab[['ReadEng_Unadj']].values

The below scalar variables will be useful later.

In [ ]:
# Number of fixed effects parameters p
p = X.shape[1]

# Number of observations
n = X.shape[0]

# Convergence tolerance
tol = 1e-6

# Set ReML
reml=True

## Calculate Kinship matrices and random effects variables

In the below code the random effects variables $\{l_k\}_{k\in\{1,...,r\}}$ (i.e. `nlevels` in the code) and $\{q_k\}_{k\in\{1,...,r\}}$  (i.e. `nraneffs` in the code) are calculated. Also calculated are the Kinship matrices, $\{\mathbf{K}^a_k\}_{k\in\{1,...,r\}}$ and $\{\mathbf{K}^c_k\}_{k\in\{1,...,r\}}$ (see Appendix 6.7 of "A Fisher Scoring approach for crossed multiple-factor Linear Mixed Models" for more information on Kinship matrices).

In [ ]:
# Number of levels and random effects for each factor
nlevels = np.zeros(r)
nraneffs = np.zeros(r)

# Dictionary to store Kinship matrices
KinshipA = dict()
KinshipC = dict()

# Loop through each family type (these are our factors)
for k in np.arange(r):

    # Record the family structure, if we haven't already.
    if k not in KinshipA:

        # Work out which family type we're looking at
        uniqueType = UniqueFamilyTypes[k]
        familyTypeTable = newTab[newTab['familyType']==uniqueType]

        # Read in the first family in this category
        uniqueFamilyIDs = np.unique(familyTypeTable[['familyID']])
        famID = uniqueFamilyIDs[0]
        famTable = familyTypeTable[familyTypeTable['familyID']==famID]

        # Work out how many subjects in family
        numSubs = len(famTable)

        # Initialize empty K_A and fill K_C with ones
        KinshipA[k] = np.zeros((numSubs,numSubs))
        KinshipC[k] = np.ones((numSubs,numSubs))

        # Loop through each pair of subjects (the families are very 
        # small in the HCP dataset so it doesn't matter if this
        # code is a little inefficient)
        for i in np.arange(numSubs):
            for j in np.arange(numSubs):
                # Check if subject i and subject j are the same person
                if i==j:
                    # In this case cov_A(i,j)=1
                    KinshipA[k][i,j]=1

                # Check if subject i and subject j are the MZ twins
                elif (famTable['ZygosityGT'].iloc[i]=='MZ' or famTable['ZygositySR'].iloc[i]=='MZ') and (famTable['ZygosityGT'].iloc[j]=='MZ' or famTable['ZygositySR'].iloc[j]=='MZ'):
                    # In this case cov_A(i,j)=1
                    KinshipA[k][i,j]=1

                # Check if subject i and subject j are full siblings (DZ is grouped into this usecase)
                elif (famTable['Mother_ID'].iloc[i]==famTable['Mother_ID'].iloc[j] and famTable['Father_ID'].iloc[i]==famTable['Father_ID'].iloc[j]):

                    # In this case cov_A(i,j)=1/2
                    KinshipA[k][i,j]=1/2

                # Check if subject i and subject j are half siblings
                elif (famTable['Mother_ID'].iloc[i]==famTable['Mother_ID'].iloc[j] or famTable['Father_ID'].iloc[i]==famTable['Father_ID'].iloc[j]):

                    # In this case cov_A(i,j)=1/2
                    KinshipA[k][i,j]=1/4

                # Else they aren't related
                else:

                    # In this case cov_A(i,j)=0
                    KinshipA[k][i,j]=0

        # Work out nlevels
        nlevels[k]=len(uniqueFamilyIDs)
        
        # Work out nraneffs
        nraneffs[k]=numSubs

# Change to ints
nlevels = np.array(nlevels, dtype=np.int32)
nraneffs = np.array(nraneffs, dtype=np.int32)

The below variables are useful for calculation. The $q$ variable matches that in the paper and is the second dimension of the $Z$ matrix. The `Dinds` variable is an array containing the indices corresponding to the first member of every family unit in the random effects design matrix.

In [ ]:
# Second dimension of Z matrix, q
q = np.sum(np.dot(nraneffs,nlevels))

# Work out D indices (there is one block of D per level)
Dinds = np.zeros(np.sum(nlevels)+1)
counter = 0
for k in np.arange(len(nraneffs)):
    for j in np.arange(nlevels[k]):
        Dinds[counter] = np.concatenate((np.array([0]), np.cumsum(nlevels*nraneffs)))[k] + nraneffs[k]*j
        counter = counter + 1

# Last index will be missing so add it
Dinds[len(Dinds)-1]=Dinds[len(Dinds)-2]+nraneffs[-1]

# Make sure indices are ints
Dinds = np.int64(Dinds)

## Constraint matrix calculation

In this code, much like in the proofs of Appendix 6.7.2, we seperate the calculation of the constraint matrix $\mathcal{C}$ into two parts; 
 - The constraint matrices mapping $\text{vec}(D_k)$ to $[\tilde{\tau}_{a,k},\tilde{\tau}_{c,k}]'$
 - The constraint matrices mapping $\tilde{\tau}=[\tilde{\tau}_{a,1},\tilde{\tau}_{c,1}, ..., \tilde{\tau}_{a,r},\tilde{\tau}_{c,r}]'$ to $\tau=[\tau_a,\tau_c]$.
 
In the code the former is referred to as the constraint matrix of the first kind and the latter referred to as the constraint matrix of the second kind (this terminology was only invented to keep track of this part of the code and is not used elsewhere).

In [ ]:
# Create constraint matrices of the first kind for mapping D_k to \sigma2_A and \sigma2_C
constrMat1stDict = dict()

# Loop through each family type and get the constraint matrix
for k in np.arange(r):
    
    # Row of constraint matrix k describing \sigmaA
    SkrowA = mat2vec2D(KinshipA[k]).transpose()
    
    # Row of constraint matrix k describing \sigmaC
    SkrowC = mat2vec2D(KinshipC[k]).transpose()

    # Construct constraint matrices
    constrMat1stDict[k]=np.concatenate((SkrowA,SkrowC),axis=0)

# Work out constraint matrix of the second kind
constrMat2nd = np.concatenate((np.tile([[1,0]],r),np.tile([[0,1]],r)),axis=0)

## Calculate $\sum X_{(k,j)}' \otimes X_{(k,j)}'$

This code is only used for speeding up the Powell optimizer to ensure fair comparison.

In [ ]:
# Work out sum over j of X_(k,j) kron X_(k,j), for each k
XkXdict = dict()

# Loop through levels and factors
for k in np.arange(r):

    # Get qk
    qk = nraneffs[k]

    # Sum XkX
    XkXdict[k] = np.zeros((p**2,qk**2))

    for j in np.arange(nlevels[k]):

        # Indices for level j of factor k
        Ikj = faclev_indices2D(k, j, nlevels, nraneffs)

        # Add to running sum
        XkXdict[k] = XkXdict[k] + np.kron(X[Ikj,:].transpose(),X[Ikj,:].transpose())

## Helper functions

The below functions calculates the log likelihood of the ACE model given an estimate of the parameters of the model, alongside the variables we have calculated above. This is the function which shall be maximized by the Powell Optimizer.

In [ ]:
def llh_ACE(paramVec, X, Y, n, p, nlevels, nraneffs, Dinds, KinshipA, KinshipC, reml=False, XkXdict=None):

    # Reshape to 2D
    paramVec = paramVec.reshape(p+3,1)

    # Get current parameter estimates
    beta = paramVec[0:p,:]
    sigma2 = paramVec[p,:][0]**2
    tau2 = paramVec[(p+1):,:]**2

    # Obtain residual vector
    e = Y - X @ beta

    # Inital D (dict version)
    Ddict = dict()
    for k in np.arange(len(nraneffs)):
        # Construct D using sigma^2A and sigma^2D
        Ddict[k] = forceSym2D(tau2[0,0]*KinshipA[k] + tau2[1,0]*KinshipC[k])

    # ------------------------------------------------------------------------------
    # Obtain (I+D)^{-1}
    # ------------------------------------------------------------------------------
    invIplusDdict = dict()
    for k in np.arange(len(nraneffs)):
        # Construct D using sigma^2A and sigma^2D
        invIplusDdict[k] = forceSym2D(np.linalg.pinv(np.eye(nraneffs[k])+Ddict[k]))


    # (D+I)^{-1} (matrix version)
    invIplusD = scipy.sparse.lil_matrix((n,n))
    counter = 0
    for k in np.arange(len(nraneffs)):
        for j in np.arange(nlevels[k]):

            # Add a block for each level of each factor.
            invIplusD[Dinds[counter]:Dinds[counter+1], Dinds[counter]:Dinds[counter+1]] = invIplusDdict[k]
            counter = counter + 1

    # Update e'V^(-1)e
    etinvVe = e.transpose() @ invIplusD @ e

    # Work out log|V| using the fact V is block diagonal
    logdetV = 0
    for k in np.arange(len(nraneffs)):
        logdetV = logdetV - nlevels[k]*np.log(np.linalg.det(invIplusDdict[k]))

    # Work out the log likelihood
    llhcurr = 0.5*(n*np.log(sigma2)+(1/sigma2)*etinvVe + logdetV)

    if reml:

        # Work out X'V^(-1)X as matrix reshape of (sum over k of ((sum_j X_(k,j) kron X_(k,j))vec(D_k)))
        XtinvVX = np.zeros((p,p))

        # Loop through levels and factors
        for k in np.arange(r):

            XtinvVX = XtinvVX + vec2mat2D(XkXdict[k] @ mat2vec2D(invIplusDdict[k]),shape=np.array([p,p]))

        logdet = np.linalg.slogdet(XtinvVX)
        llhcurr = llhcurr - 0.5*logdet[0]*logdet[1] + 0.5*p*np.log(sigma2)

    return(llhcurr)


The below function calculates the standard error of $\hat{\beta}$ for the ACE model.

In [ ]:
def seBeta_ACE(paramVec, p, KinshipA, KinshipC, nlevels, nraneffs):

    # Work out beta, sigma2 and the vector of variance components
    beta = paramVec[0:p,:]
    sigma2 = paramVec[p,0]**2
    tau2 = paramVec[(p+1):,:]**2/sigma2

    # Get D in dictionary form
    Ddict = dict()
    for k in np.arange(len(nraneffs)):
        # Construct D using sigma^2A and sigma^2D
        Ddict[k] = tau2[0,0]*KinshipA[k] + tau2[1,0]*KinshipC[k]

    # r, total number of random factors
    r = len(nlevels)

    # Work out sum over j of X_(k,j) kron X_(k,j), for each k
    XkXdict = dict()

    # Loop through levels and factors
    for k in np.arange(r):

        # Get qk
        qk = nraneffs[k]

        # Sum XkX
        XkXdict[k] = np.zeros((p**2,qk**2))

        for j in np.arange(nlevels[k]):

            # Indices for level j of factor k
            Ikj = faclev_indices2D(k, j, nlevels, nraneffs)

            # Add to running sum
            XkXdict[k] = XkXdict[k] + np.kron(X[Ikj,:].transpose(),X[Ikj,:].transpose())

            
    # Work out X'V^(-1)X as matrix reshape of (sum over k of ((sum_j X_(k,j) kron X_(k,j))vec(D_k)))
    XtinvVX = np.zeros((p,p))

    # Loop through levels and factors
    for k in np.arange(r):

        XtinvVX = XtinvVX + vec2mat2D(XkXdict[k] @ mat2vec2D(np.linalg.pinv(np.eye(nraneffs[k])+Ddict[k])),shape=np.array([p,p]))


    # Get variance of beta
    varb = sigma2*np.linalg.inv(XtinvVX)

    return(np.sqrt(np.diagonal(varb)))

## Initial Estimates

The below code generates some initial estimates of $\beta$, $\sigma^2$ and $\tau$. These shall be used later on by the Powell optimizer only.

In [ ]:
# ------------------------------------------------------------------------------
# Product matrices (only used here)
# ------------------------------------------------------------------------------
XtX = X.transpose() @ X
XtY = X.transpose() @ Y
YtY = Y.transpose() @ Y
YtX = Y.transpose() @ X

# ------------------------------------------------------------------------------
# Initial estimates
# ------------------------------------------------------------------------------
# If we have initial estimates use them.

# Inital beta
beta = initBeta2D(XtX, XtY)

# Work out e'e
ete = ssr2D(YtX, YtY, XtX, beta)

# Initial sigma2
sigma2 = initSigma22D(ete, n)
sigma2 = np.maximum(sigma2,1e-10) # Prevent hitting boundary
sigma2 = np.array([[sigma2]])

# Initial zero matrix to hold the matrices Ckcov(dl/Dk)Ck'
FDk = np.zeros((2*r,2*r))

# Initial zero vector to hold the vectors Ck*dl/dDk
CkdldDk = np.zeros((2*r,1))

# Initial residuals
e = Y - X @ beta
eet = e @ e.transpose()

for k in np.arange(r):

    # Get FDk
    FDk[2*k:(2*k+2),2*k:(2*k+2)]= nlevels[k]*constrMat1stDict[k] @ constrMat1stDict[k].transpose()

    # Initialize empty sum
    eetSum = np.zeros((nraneffs[k],nraneffs[k]))

    # Get sum ee'_[k,j,j]
    for j in np.arange(nlevels[k]):

        # Get indices for current block
        Ikj = faclev_indices2D(k, j, nlevels, nraneffs)

        # Add to current sum
        eetSum = eetSum + eet[np.ix_(Ikj,Ikj)]

    # Get Ck*dl/dDk
    CkdldDk[2*k:(2*k+2),:] = constrMat1stDict[k] @ mat2vec2D(nlevels[k]-eetSum/sigma2)


# Initial vec(sigma^2A/sigma^2E, sigma^2C/sigma^2E)
tau2 = np.linalg.pinv(constrMat2nd @ FDk @ constrMat2nd.transpose()) @ constrMat2nd @ CkdldDk

# Initial parameter vector
initParams = np.concatenate((beta, sigma2, tau2*sigma2))

## PFS method

The below code runs the PFS algorithm and times it.

In [ ]:
# Run and time the PFS algorithm
t1 = time.time()
PFS_out=pFS_ACE2D(X, Y, nlevels, nraneffs, tol, n, KinshipA, KinshipC, constrMat1stDict, constrMat2nd,reml=reml)
t2 = time.time()

Below is the time taken for computation for the PFS method.

In [ ]:
print('Computation time: ', t2-t1)

Below are the estimates for the $\beta$ parameters obtained with the PFS method.

In [ ]:
# Get the parameter vector
paramVecACE = np.array(PFS_out[0])

# Convert it to display format
toDisplay = np.array(paramVecACE)
toDisplay[(p+1):,:] = toDisplay[(p+1):,:]*toDisplay[p,0]

# Print beta estimates
print('Beta 0 (intercept):  ', toDisplay[0,0])
print('Beta 1 (Age):        ', toDisplay[1,0])
print('Beta 2 (Sex):        ', toDisplay[2,0])
print('Beta 3 (Age:Sex):    ', toDisplay[3,0])
print('Beta 4 (PSQI score): ', toDisplay[4,0])

Below are the estimates for the $\sigma^2$ parameters obtained with the PFS method.

In [ ]:
# Print sigma^2 estimates
print('Sigma^2 A (Additive genetic):   ', toDisplay[6,0]**2)
print('Sigma^2 C (Common Environment): ', toDisplay[7,0]**2)
print('Sigma^2 E (Error):              ', toDisplay[5,0]**2)

Below is the maximized log-likelihood output by the PFS method:

In [ ]:
print('log-likelihood: ', -(llh_ACE(paramVecACE, X, Y, n, p, nlevels, nraneffs, Dinds, KinshipA, KinshipC, reml=reml, XkXdict=XkXdict)[0,0]-n/2*np.log(2*np.pi)))

Below are the standard errors for the $\beta$ estimates:

In [ ]:
seBeta = seBeta_ACE(toDisplay, p, KinshipA, KinshipC, nlevels, nraneffs)
print('Standard Errors for beta estimates: ', seBeta)

Below are the results for the T tests:

In [ ]:
contrastNames = ['Intercept', 'Age', 'Sex', 'Age:Sex', 'PSQI Score']

# Loop through fixed effects parameters
for j in np.arange(p):
    
    print('-----------------------------------------')
    print('T test for contrast ' + str(j) + ': ' + contrastNames[j])
    print('-----------------------------------------')
    
    # Create contrast vectors
    L = np.zeros((1,p))
    L[0,j]=1

    # Obtain the satterthwaithe degrees of freedom estimate for the contrast
    swdf = get_swdf_ACE_T2D(L, paramVecACE, X, nlevels, nraneffs, KinshipA, KinshipC, constrMat1stDict)
    
    print('swdf: ', swdf[0,0])
    
    # Obtain the T statistic for the contrast
    T = get_T_ACE_2D(L, X, paramVecACE, KinshipA, KinshipC, nlevels, nraneffs)
    print('T: ', T[0,0])
    
    # Obtain the P value for the T statistic
    if T < 0:
        pvalACE = 1-stats.t.cdf(T, swdf)
    else:
        pvalACE = stats.t.cdf(-T, swdf)

    # Convert the T test between two tailed and one tailed format.
    if pvalACE < 0.5:
        pvalACE = 2*pvalACE
    else:
        pvalACE = 2*(1-pvalACE)

    print('P: ', pvalACE[0,0])


## Powell method

The below code runs the Powell optimizer and times it.

In [ ]:
t1 = time.time()
Powell_out = minimize(llh_ACE, initParams, args=(X, Y, n, p, nlevels, nraneffs, Dinds, KinshipA, KinshipC, reml, XkXdict), method='Powell', tol=1e-6)
t2 = time.time()

Below is the time taken for computation for the Powell method.

In [ ]:
print('Computation time: ', t2-t1)

Below are the estimates for the $\beta$ parameters obtained with the Powell method.

In [ ]:
# Get the parameter vector
paramVecOpt = Powell_out['x'].reshape((p+3),1)

# Convert it to display format
toDisplay = np.array(paramVecACE)
toDisplay[(p+1):,:] = toDisplay[(p+1):,:]*toDisplay[p,0]

# Print beta coordinates
print('Beta 0 (intercept):  ', toDisplay[0,0])
print('Beta 1 (Age):        ', toDisplay[1,0])
print('Beta 2 (Sex):        ', toDisplay[2,0])
print('Beta 3 (Age:Sex):    ', toDisplay[3,0])
print('Beta 4 (PSQI score): ', toDisplay[4,0])

Below are the estimates for the $\sigma^2$ parameters obtained with the PFS method.

In [ ]:
# Print sigma^2 estimates
print('Sigma^2 A (Additive genetic):   ', toDisplay[6,0]**2)
print('Sigma^2 C (Common Environment): ', toDisplay[7,0]**2)
print('Sigma^2 E (Error):              ', toDisplay[5,0]**2)

Below is the maximized log-likelihood output by the Powell method:

In [ ]:
print(-(np.array([[Powell_out['fun']]])[0,0]-n/2*np.log(2*np.pi)))

Below are the standard errors for the $\beta$ estimates:

In [ ]:
seBeta = seBeta_ACE(toDisplay, p, KinshipA, KinshipC, nlevels, nraneffs)
print('Standard Errors for beta estimates: ', seBeta)

Below are the results for the T tests:

In [ ]:
# Loop through fixed effects parameters
for j in np.arange(p):
    
    print('-----------------------------------------')
    print('T test for contrast ' + str(j) + ': ' + contrastNames[j])
    print('-----------------------------------------')
    
    # Create contrast vectors
    L = np.zeros((1,p))
    L[0,j]=1
    
    # Obtain the satterthwaithe degrees of freedom estimate for the contrast
    swdf = get_swdf_ACE_T2D(L, paramVecOpt, X, nlevels, nraneffs, KinshipA, KinshipC, constrMat1stDict)
    
    print('swdf: ', swdf[0,0])
    
    # Obtain the T statistic for the contrast
    T = get_T_ACE_2D(L, X, paramVecOpt, KinshipA, KinshipC, nlevels, nraneffs)
    
    print('T: ', T[0,0])
    
    # Obtain the P value for the T statistic
    if T < 0:
        pvalOpt = 1-stats.t.cdf(T, swdf)
    else:
        pvalOpt = stats.t.cdf(-T, swdf)

    # Convert the T test between two tailed and one tailed format.
    if pvalOpt < 0.5:
        pvalOpt = 2*pvalOpt
    else:
        pvalOpt = 2*(1-pvalOpt)

    print('P: ', pvalOpt[0,0])

## OLS method

The below code runs OLS and times it.

In [ ]:
# Start the clock
t1 = time.time()

# Obtain beta estimates
betaOLS = np.linalg.pinv(X.transpose() @ X) @ X.transpose() @ Y

# Obtain residual vector
e = Y - X @ betaOLS

# Obtain sigma_OLS
sigmaOLS = np.sqrt(e.transpose() @ e/(n-p))

# stop the clock
t2 = time.time()

# Reformat parameters
sigma2OLS = sigmaOLS**2

# Construct parameter vector
paramVecOLS = np.zeros(((p+3),1))
paramVecOLS[0:p,:] = betaOLS
paramVecOLS[p,:] = sigmaOLS[0,0]

# Display variable
toDisplay = np.array(paramVecOLS)

Below is the time taken for OLS computation.

In [ ]:
print('Computation time: ', t2-t1)

Below are the estimates for the $\beta$ parameters obtained with OLS.

In [ ]:
# Print beta coordinates
print('Beta 0 (intercept):  ', toDisplay[0,0])
print('Beta 1 (Age):        ', toDisplay[1,0])
print('Beta 2 (Sex):        ', toDisplay[2,0])
print('Beta 3 (Age:Sex):    ', toDisplay[3,0])
print('Beta 4 (PSQI score): ', toDisplay[4,0])

Below are the estimates for the $\sigma^2$ parameters obtained with OLS.

In [ ]:
# Print sigma^2 estimates
print('Sigma^2 A (Additive genetic):   ', toDisplay[6,0]**2)
print('Sigma^2 C (Common Environment): ', toDisplay[7,0]**2)
print('Sigma^2 E (Error):              ', toDisplay[5,0]**2)

Below is the maximized log-likelihood output by OLS:

In [ ]:
print(-(llh_ACE(toDisplay, X, Y, n, p, nlevels, nraneffs, Dinds, KinshipA, KinshipC, reml=reml, XkXdict=XkXdict)[0,0]-n/2*np.log(2*np.pi)))

Below are the standard errors for the $\beta$ estimates:

In [ ]:
seBeta = seBeta_ACE(toDisplay, p, KinshipA, KinshipC, nlevels, nraneffs)
print('Standard Errors for beta estimates: ', seBeta)

Below are the results for the T tests:

In [ ]:
# Loop through fixed effects parameters
for i in np.arange(X.shape[1]):
    
    print('-----------------------------------------')
    print('T test for contrast ' + str(j) + ': ' + contrastNames[j])
    print('-----------------------------------------')

    # Create contrast vectors
    L = np.zeros((1,X.shape[1]))
    L[0,i]=1

    # Obtain the T statistic for the contrast
    TOLS = (L @ betaOLS)/ np.sqrt(sigma2OLS*(L @ np.linalg.pinv(XtX) @ L.transpose()))

    # Degrees of freedom for OLS
    df_OLS = n-p

    # Obtain the P value for the T statistic
    if T < 0:
        pvalOLS = 1-stats.t.cdf(TOLS, df_OLS)
    else:
        pvalOLS = stats.t.cdf(-TOLS, df_OLS)

    # Convert the T test between two tailed and one tailed format.
    if pvalOLS < 0.5:
        pvalOLS = 2*pvalOLS
    else:
        pvalOLS = 2*(1-pvalOLS)

    print('df: ', df_OLS)
    print('T: ', TOLS[0,0])
    print('pval: ', pvalOLS[0,0])